In [2]:
# imports

from typing import TypedDict
from langgraph.graph import StateGraph, END, START

In [6]:
class DiscountState(TypedDict):
    total : float
    discount_rate: float
    final_price:float
    

In [7]:
def check_eligibility(state: DiscountState):
    total = state["total"]
    if total > 1000:
        return {'discount_rate': 0.20}
    elif total > 500:
        return {'discount_rate': 0.10}
    else:
        return {'discount_rate': 0.00}

In [8]:
def apply_discount(state: DiscountState):
    total = state["total"]
    rate = state["discount_rate"]
    final = total *( 1- rate)
    return {'final_price': final}

In [9]:
def no_discount(state: DiscountState):
    return {'final_price': state["total"]}

In [10]:
def router_discount(state: DiscountState):
    if state["discount_rate"] > 0:
        return "apply"
    else:
        return "no"

In [17]:
graph = StateGraph(DiscountState)

graph.add_node("apply_discount", apply_discount)
graph.add_node("check_eligibility", check_eligibility)
graph.add_node("no_discount", no_discount)

graph.add_edge(START, "check_eligibility")

#graph.add_edge("check_eligibility","apply_discount")
#graph.add_edge("check_eligibility","no_discount")
graph.add_conditional_edges("check_eligibility", router_discount,{
    "apply": "apply_discount",
    "no": "no_discount"
})


graph.add_edge("apply_discount", END)

graph.add_edge("no_discount", END)



In [19]:
workflow = graph.compile()

In [24]:
response = workflow.invoke({"total": 300})

In [25]:
print(response)

{'total': 300, 'discount_rate': 0.0, 'final_price': 300}
